# <u> Problema de los ascensores</u> (Primera Versión)


### G05 - Boris Carballa Corredoira, Juan Carlos Villanueva Quirós, Francisco Javier Blázquez Martínez

Primera aproximación al problema, el edificio consta de un único bloque con un único ascensor.

In [40]:
from search import *
import copy
import time

#### Constantes
+ *CAP_ASCENSOR*: Capacidad de los ascensores
+ *EN_ASCENSOR*: Nos indica que una persona se encuentra dentro de un ascensor
+ *NINGUNA*: Usada para indicar que una lista es vacia
+ *EN_DESTINO*: Denota que una persona ha llegado a su destino

In [41]:
CAP_ASCENSOR = 2
EN_ASCENSOR = -1
NINGUNA = -1
EN_DESTINO = -2

#### Solution Manager
Importamos el notebook *SolutionManager* en el cual tenemos almacenados todos los procedimientos que se ejecutan una vez calculada la solución para mejorar la visualización de la misma.

In [42]:
%run SolutionManager.ipynb

#### Clase NodeState

Clase que representa un nodo del espacio de exploración. El estado de dicho nodo se compone de:

 + *personas*: Lista con la planta en la que se encuentra cada persona, si una persona se encuentra dentro del ascensor toma el  valor de la constante EN_ASCENSOR, si se encuentra en su destino toma el valor de la costante EN_DESTINO
 + *ascensor_pos*: Planta en la que se encuentra el ascensor
 + *ascensor_personas*: Lista con las personas (índices de la lista "personas") que se encuentran dentro del ascensor
 
Por otro lado, hemos definido los operadores igual (**eq**), menor que (**lt**) y **hash** para permitir la comparación entre nodos y posibilitar así la correcta inserción de estos en estructuras de datos para que las búsquedas usen control de repetidos.

In [43]:
class NodeState:

    def __init__(self, init_personas):
        self.personas = init_personas
        self.ascensor_pos = 0
        self.ascensor_personas = []
        
    def __eq__(self, nodo):
        return self.personas == nodo.personas and self.ascensor_pos == nodo.ascensor_pos and self.ascensor_personas == nodo.ascensor_personas
    
    # No hemos definido un orden en los nodos, sin embargo, diversos algoritmos 
    # requieren tener este métdodo implementado.
    def __lt__(self,nodo):
        return True
    
    def __hash__(self):
        return hash((tuple(self.personas), self.ascensor_pos, tuple(self.ascensor_personas)))

#### Clase AscensoresSuperSimplificado
Clase que representa el problema en su primera versión (un único bloque con un único ascensor). Consta de varias funciones:

+ **Init**: Dado un nodo inicial y un nodo objetivo, inicializa la clase

    + *initial*: nodo desde el que empezamos a buscar
    + *goal*: Plantas a las que quieren llegar las respectivas personas
    + *analizados*: número de nodos analizados hasta el momento

+ **Actions**: Dado un nodo, devuelve las acciones posibles que podemos ejecutar desde dicho nodo. Las acciones posibles son: subir, bajar,coger_persona y dejar_persona.
 
    + *Codificación Acciones*. Hemos decidido codificar las acciones posibles de la siguiente manera:
        + (subir, i, j) &rArr; El ascensor i sube j plantas
        + (bajar, i, j) &rArr; El ascensor i baja j plantas
        + (coger_persona, i, j) &rArr; El ascensor i coge a la persona j
        + (dejar_persona, i, j) &rArr; El ascesnro i deja a la persona j
      
    + *Variables locales*:
        + personas_esta_planta_suben: Lista de las personas que se encuentran en la planta del ascensor y su planta objetivo está por encima
        + personas_esta_planta_bajan: Lista de las personas que se encuentran en la planta del ascensor y su planta objetivo está por debajo
        + persona_menos_arriba: Menor planta de las personas que se encuentran por encima del ascensor
        + persona_menos_abajo: Mayor planta de las personas que se encuentran por debajo del ascensor
        + destino_menos_arriba: Menor destino de las personas que se encuentran dentro del ascensor
        + destino_menos_abajo: Mayor destino de las personas que se encuentran dentro del ascensor
        
    + *Restricciones*. Hemos decidido implementar una serie de restricciones basadas en invariantes implicitos del problema que nos permiten no dar una acción cuando sea innecesario (no nos aporta un camino hacia la solución óptima) eliminando ramas sin perder optimalidad. Todas estas restricciones han sido probadas válidas en las hipótesis del problema (de las cuales cabe destacar la no penalización por parada de los ascensores) por el método de reducción de diferencias:
        1. Un ascensor nunca tiene dentro personas que quieran subir y bajar. O bien tiene dentro a gente que quiere bajar, o bien a gente que quiere subir. De esta forma, evitamos alejar de su objetivo innecesariamente a las personas que quieren bajar cuando el ascensor sube y viceversa
        2. Si alguien llega a su planta objetivo, forzamos a que esta baje del ascensor, es decir, devolvemos como única acción posible dejar a dicha persona. Siempre será más óptimo dejar a la persona una vez alcance su objetivo y no alejarla de este
        3. Solo subimos cuando haya una persona esperando en una planta por encima del ascensor o cuando haya alguna persona dentro del ascensor cuya planta objetivo esté por encima. De esta manera, nos evitamos dar la opción de subir cuando no aporte nada al problema. (Respectivamente con bajar)
        4. No subimos cuando haya un hueco en el ascensor y alguien en la planta que quiera subir. De esta forma, obligamos a coger a esa persona antes de subir pues siempre será más eficiente acercarla si es posible. (Respectivamente con bajar)

+ **Result**. Dado un nodo y una acción, devuelve el resultado de aplicar al nodo dicha acción

+ **Goal_test**. Dado un nodo, comprueba si hemos alcanzado la solución. Habremos alcanzado el objetivo cuando todas las personas tengan el valor EN_DESTINO

+ **path_cost**. Si la acción que aplicamos es *subir* o *bajar*, incrementamos el coste en el número de plantas que subimos o bajamos. El coste total será la suma de las plantas que subimos  o bajamos

+ **h**. Heurística a usar en la búsqueda *A\**. Hemos elegido para esta primera versión que sea la suma para cada persona de las diferencias entre la planta en la que se encuentra y la planta objetivo dividido esto entre la constante CAP_ASCENSOR (ya que únicamente contamos con un ascensor). La heurística es admisible y consistente (h(s1) <= c(s1,a,s2) + h(s2)). No dividir por esta constante implica usar una heurística no consistente (y por tanto no llegar a solución óptima necesariamente) pero obtener soluciones en un tiempo de ejecución menor.

In [81]:
class AscensoresSuperSimplificado(Problem) :    
   
    def __init__(self, initial, goal=None):        
        self.initial = initial
        self.goal = goal
        self.analizados = 0

    def actions(self, state):
        
        num_ascensor = 0
        personas          = state.personas
        ascensor_pos      = state.ascensor_pos
        ascensor_personas = state.ascensor_personas
        ascensor_vacio    = not ascensor_personas
        ascensor_lleno    = (len(ascensor_personas)==CAP_ASCENSOR)
        
        personas_esta_planta_suben = [i for i in range(len(personas)) if personas[i] == ascensor_pos and self.goal[i] > ascensor_pos]
        personas_esta_planta_bajan = [i for i in range(len(personas)) if personas[i] == ascensor_pos and self.goal[i] < ascensor_pos]
    
        # 1.- Si alguien ha llegado a su destino, forzamos que baje del ascensor, 
        #     devolviendo como unica accion posible DEJAR PERSONA. (Restriccion 2)
        for persona in ascensor_personas:
            if ascensor_pos==self.goal[persona]:
                return [(DEJAR_PERSONA,num_ascensor,persona)]
        
        
        accs = list()
        
        # 2.- SUBIR:
        
        persona_menos_arriba = min([person for person in personas if person>ascensor_pos], default=NINGUNA)
        
        # Si el ascensor está vacío sólo subimos si hay alguna persona por arriba 
        # y no hay gente en esta planta que quiera subir (Restriccion 3 y 4)
        if ascensor_vacio:
            if persona_menos_arriba!=NINGUNA and not(personas_esta_planta_suben):
                accs.append((SUBIR,num_ascensor,persona_menos_arriba-ascensor_pos))
        
        # Si el ascensor no está vacío, comprobamos que sus personas suben y vamos
        # al mínimo entre a donde suben estas y donde hay una persona por encima
        # siempre que no nos quede espacio o no haya gente en esa planta que quiere subir
        # en cuyo caso cogeremos a dichar persona para acercarla lo maximo posible (Restriccion 4)
        elif self.goal[ascensor_personas[0]]>ascensor_pos:
            destino_menos_arriba = min([self.goal[i] for i in ascensor_personas])
            
            if ascensor_lleno or not(personas_esta_planta_suben):
                if persona_menos_arriba==NINGUNA:
                    accs.append((SUBIR,num_ascensor,destino_menos_arriba-ascensor_pos))
                else:
                    accs.append((SUBIR,num_ascensor,min(persona_menos_arriba, destino_menos_arriba)-ascensor_pos))
                             
                             
        # 3.- BAJAR. La acción bajar es totalmente dual a la acción subir, mismas lógica en
        #     las condiciones. 
        
        persona_menos_abajo = max([person for person in personas if person<ascensor_pos and person!=EN_ASCENSOR and person!=EN_DESTINO], default=NINGUNA)
        
        if ascensor_vacio:
            if persona_menos_abajo!=NINGUNA and not(personas_esta_planta_bajan):
                accs.append((BAJAR,num_ascensor,ascensor_pos-persona_menos_abajo))
        elif self.goal[ascensor_personas[0]]<ascensor_pos:
            destino_menos_abajo = max([self.goal[i] for i in ascensor_personas])
            
            if ascensor_lleno  or not(personas_esta_planta_bajan):
                if persona_menos_abajo==NINGUNA:
                    accs.append((BAJAR,num_ascensor,ascensor_pos-destino_menos_abajo))
                else:
                    accs.append((BAJAR,num_ascensor,ascensor_pos-max(persona_menos_abajo, destino_menos_abajo)))
        
        # 4.- DEJAR PERSONA: Para cada persona del ascensor, podemos dejarla
        for personaInterior in ascensor_personas:
            accs.append((DEJAR_PERSONA,num_ascensor,personaInterior))
        
        # 5. COGER PERSONA. En el interior de ascensor todos suben, o bajan, no ambas (Restriccion 1)
        if not ascensor_lleno:
            #Si el ascensor esta vacio, podemos coger a cualquiera, suban o bajen
            if ascensor_vacio:
                for persona in personas_esta_planta_suben:
                    accs.append((COGER_PERSONA,num_ascensor,persona))
                for persona in personas_esta_planta_bajan:
                    accs.append((COGER_PERSONA,num_ascensor,persona))
            else:
                #Si el ascensor tiene a alguien que baja, solo cogemos a gente que baje
                if self.goal[ascensor_personas[0]]<ascensor_pos:
                    for persona in personas_esta_planta_bajan:
                        accs.append((COGER_PERSONA,num_ascensor,persona))
                else:
                #Si el ascensor tiene a alguien que sube, solo cogemos a gente que suba
                    for persona in personas_esta_planta_suben:
                        accs.append((COGER_PERSONA,num_ascensor,persona))
        
        return accs

    def result(self, state, action):
        estado_nuevo = copy.deepcopy(state)
        
        ascensor = action[1]
        ascensor_pos = state.ascensor_pos
        ascensor_personas = state.ascensor_personas
        personas = state.personas
        
        #Si bajamos, la nueva posicion del ascensor será la actual menos las plantas a bajar
        if action[0]==BAJAR:
            estado_nuevo.ascensor_pos = estado_nuevo.ascensor_pos - action[2]
        #Si subimos, la nueva posicion del ascensor será la actual más las plantas a subir
        elif action[0]==SUBIR:
            estado_nuevo.ascensor_pos = estado_nuevo.ascensor_pos + action[2]
        #Si cogemos persona, introducimos en ascensor_personas la persona que cogemos. Además indicamos en
        #personas que se encuentra EN_ASCENSOR
        elif action[0]==COGER_PERSONA:
            estado_nuevo.ascensor_personas.append(action[2])
            estado_nuevo.personas[action[2]] = EN_ASCENSOR
        #Si dejamos persona, la quitamos de ascensor_personas. Si ha llegado a su destino, ponemos EN_DESTINO
        #y si no, la nueva planta en la que se encuentra (la del ascensor)
        else:
            estado_nuevo.ascensor_personas.remove(action[2])
            
            if self.goal[action[2]]==ascensor_pos:
                estado_nuevo.personas[action[2]] = EN_DESTINO
            else:
                estado_nuevo.personas[action[2]] = ascensor_pos
        
        return estado_nuevo
    
    def goal_test(self, state):
        self.analizados +=1
        return state.personas == [EN_DESTINO for persona in state.personas]
    
    # Un único ascensor, el coste es el número de plantas que se desplaza
    def path_cost(self, c, state1, action, state2):
        if action[0]==BAJAR or action[0]==SUBIR:
            return c + action[2]
        else:
            return c
    
    # Heurística que garantiza solución óptima, eliminando la división por la
    # constante CAP_ASCENSOR obtenemos una heurística que alcanza solución 
    # (aunque no necesariamente óptima) en un menor tiempo. 
    def h(self,node):
        suma  = sum([abs(self.goal[i]-node.state.personas[i]) for i in range(0,len(node.state.personas)) if node.state.personas[i]!=EN_DESTINO and node.state.personas[i]!=EN_ASCENSOR])
        suma += sum([abs(self.goal[persona]-node.state.ascensor_pos) for persona in node.state.ascensor_personas])
        return suma/CAP_ASCENSOR
        
    def value(self, state):
        raise NotImplementedError

#### Probando nuestra primera versión
Inicializamos el nodo init con las plantas donde se encuentran al comienzo cada persona y la lista goal con las plantas a las que quieren ir. Declaramos el problema con dichos nodos y ejecutamos la búsqueda. Posteriormente, decodificamos la lista de acciones obtenidas (Módulo SolutionManager) para visualizarla cómodamente con una interpretación y no como tuplas codificadas.

Como podemos observar, logramos hallar una solución al problema para una gran cantidad de personas en un tiempo muy disminuido.

In [46]:
#Caso sencillo
init = NodeState([0,1])
goal = [1,0]
problem = AscensoresSuperSimplificado(init, goal)
start = time.time()
acciones = astar_search(problem).solution()
end = time.time()
decodificador_acciones(fusionador_acciones(acciones))

El ascensor 0 coge a la persona 0
El ascensor 0 sube 1 planta
El ascensor 0 deja a la persona 0
El ascensor 0 coge a la persona 1
El ascensor 0 baja 1 planta
El ascensor 0 deja a la persona 1


In [53]:
print("Nodos analizados: " + str(problem.analizados))
print("Coste secuencial: " + str(coste_acciones(acciones)))
print("Tiempo de búsqueda: " + str(end-start) + " s")

Nodos analizados: 5934
Coste secuencial: 34
Tiempo de búsqueda: 2.7810983657836914 s


In [48]:
#Caso del enunciado
init = NodeState([2,4,1,8,1])
goal = [3,11,12,1,9]
problem = AscensoresSuperSimplificado(init, goal)
start = time.time()
acciones = astar_search(problem).solution()
end = time.time()
decodificador_acciones(fusionador_acciones(acciones))

El ascensor 0 sube 1 planta
El ascensor 0 coge a las personas 4, 2
El ascensor 0 sube 1 planta
El ascensor 0 deja a la persona 4
El ascensor 0 coge a la persona 4
El ascensor 0 sube 6 plantas
El ascensor 0 deja a la persona 2
El ascensor 0 coge a la persona 2
El ascensor 0 sube 1 planta
El ascensor 0 deja a las personas 4, 2
El ascensor 0 baja 1 planta
El ascensor 0 coge a la persona 3
El ascensor 0 baja 7 plantas
El ascensor 0 deja a la persona 3
El ascensor 0 sube 1 planta
El ascensor 0 coge a la persona 0
El ascensor 0 sube 1 planta
El ascensor 0 deja a la persona 0
El ascensor 0 sube 1 planta
El ascensor 0 coge a la persona 1
El ascensor 0 sube 5 plantas
El ascensor 0 coge a la persona 2
El ascensor 0 sube 2 plantas
El ascensor 0 deja a la persona 1
El ascensor 0 sube 1 planta
El ascensor 0 deja a la persona 2


In [52]:
print("Nodos analizados: " + str(problem.analizados))
print("Coste secuencial: " + str(coste_acciones(acciones)))
print("Tiempo de búsqueda: " + str(end-start) + " s")

Nodos analizados: 5934
Coste secuencial: 34
Tiempo de búsqueda: 2.7810983657836914 s


In [50]:
#Caso 10 personas, 12 plantas:
init = NodeState([10, 8, 1, 12, 7, 9, 5, 11, 6, 2])
goal = [7, 3, 4, 5, 6, 10, 2, 4, 9, 7]
problem = AscensoresSuperSimplificado(init, goal)
start = time.time()
acciones = astar_search(problem).solution()
end = time.time()
decodificador_acciones(fusionador_acciones(acciones))

El ascensor 0 sube 1 planta
El ascensor 0 coge a la persona 2
El ascensor 0 sube 1 planta
El ascensor 0 coge a la persona 9
El ascensor 0 deja a la persona 2
El ascensor 0 coge a la persona 2
El ascensor 0 sube 2 plantas
El ascensor 0 deja a la persona 2
El ascensor 0 sube 2 plantas
El ascensor 0 coge a la persona 8
El ascensor 0 deja a la persona 9
El ascensor 0 coge a la persona 9
El ascensor 0 sube 1 planta
El ascensor 0 deja a la persona 9
El ascensor 0 sube 2 plantas
El ascensor 0 deja a la persona 8
El ascensor 0 coge a la persona 5
El ascensor 0 sube 1 planta
El ascensor 0 deja a la persona 5
El ascensor 0 sube 2 plantas
El ascensor 0 coge a la persona 3
El ascensor 0 baja 1 planta
El ascensor 0 coge a la persona 7
El ascensor 0 baja 6 plantas
El ascensor 0 deja a la persona 3
El ascensor 0 coge a la persona 6
El ascensor 0 deja a la persona 7
El ascensor 0 coge a la persona 7
El ascensor 0 baja 1 planta
El ascensor 0 deja a las personas 7, 6
El ascensor 0 sube 6 plantas
El asce

In [54]:
print("Nodos analizados: " + str(problem.analizados))
print("Coste secuencial: " + str(coste_acciones(acciones)))
print("Tiempo de búsqueda: " + str(end-start) + " s")

Nodos analizados: 5934
Coste secuencial: 34
Tiempo de búsqueda: 2.7810983657836914 s


#### Casos de alta complejidad:

Los casos siguientes resultan demasiado lentos para la heurística consistente. Sin embargo si renunciamos a obtener siempre la solución óptima (en términos de minimizar el número de plantas que tiene que desplazarse) obtenemos soluciones muy rápidas. Se debe usar la siguiente heurística:

In [65]:
def h(node):
    suma  = sum([abs(goal[i]-node.state.personas[i]) for i in range(0,len(node.state.personas)) if node.state.personas[i]!=EN_DESTINO and node.state.personas[i]!=EN_ASCENSOR])
    suma += sum([abs(goal[persona]-node.state.ascensor_pos) for persona in node.state.ascensor_personas])
    return suma

In [74]:
#Caso 15 personas, 20 plantas:
init = NodeState([10, 8, 1, 12, 7, 9, 5, 11, 6, 2, 4, 18, 2, 7, 5])
goal = [7, 3, 12, 5, 6, 10, 2, 4, 9, 11, 16, 2, 11, 12, 18]
problem = AscensoresSuperSimplificado(init, goal)
start = time.time()
acciones = astar_search(problem,h).solution()
end = time.time()
decodificador_acciones(fusionador_acciones(acciones))

El ascensor 0 sube 1 planta
El ascensor 0 coge a la persona 2
El ascensor 0 sube 1 planta
El ascensor 0 coge a la persona 12
El ascensor 0 sube 9 plantas
El ascensor 0 deja a las personas 12, 2
El ascensor 0 coge a la persona 7
El ascensor 0 baja 1 planta
El ascensor 0 coge a la persona 0
El ascensor 0 baja 3 plantas
El ascensor 0 deja a la persona 0
El ascensor 0 coge a la persona 4
El ascensor 0 baja 1 planta
El ascensor 0 deja a las personas 4, 7
El ascensor 0 coge a la persona 8
El ascensor 0 sube 1 planta
El ascensor 0 coge a la persona 13
El ascensor 0 sube 2 plantas
El ascensor 0 deja a la persona 8
El ascensor 0 coge a la persona 5
El ascensor 0 sube 1 planta
El ascensor 0 deja a la persona 5
El ascensor 0 sube 1 planta
El ascensor 0 coge a la persona 2
El ascensor 0 sube 1 planta
El ascensor 0 deja a las personas 13, 2
El ascensor 0 coge a la persona 3
El ascensor 0 baja 4 plantas
El ascensor 0 coge a la persona 1
El ascensor 0 baja 3 plantas
El ascensor 0 deja a la persona 3


In [75]:
print("Nodos analizados: " + str(problem.analizados))
print("Coste secuencial: " + str(coste_acciones(acciones)))
print("Tiempo de búsqueda: " + str(end-start) + " s")

Nodos analizados: 93
Coste secuencial: 80
Tiempo de búsqueda: 0.016496658325195312 s


In [79]:
#Caso 30 personas, 100 plantas:
init = NodeState([34, 12, 25, 51, 30, 83, 29, 3, 17, 66, 47, 89, 92, 0, 71, 35, 96, 69, 6, 40, 67, 93, 24, 97, 63, 79, 19, 62, 99, 27])
goal = [21, 92, 18, 28, 35, 68, 74, 81, 89, 27, 24, 70, 79, 61, 8, 91, 7, 63, 37, 99, 72, 58, 59, 73, 25, 66, 5, 86, 34, 83]
problem = AscensoresSuperSimplificado(init, goal)
start = time.time()
acciones = astar_search(problem,h).solution()
end = time.time()
decodificador_acciones(fusionador_acciones(acciones))

El ascensor 0 coge a la persona 13
El ascensor 0 sube 3 plantas
El ascensor 0 coge a la persona 7
El ascensor 0 sube 58 plantas
El ascensor 0 deja a la persona 13
El ascensor 0 sube 1 planta
El ascensor 0 coge a la persona 27
El ascensor 0 sube 19 plantas
El ascensor 0 deja a la persona 7
El ascensor 0 sube 2 plantas
El ascensor 0 deja a la persona 27
El ascensor 0 coge a la persona 5
El ascensor 0 baja 4 plantas
El ascensor 0 coge a la persona 25
El ascensor 0 baja 11 plantas
El ascensor 0 deja a la persona 5
El ascensor 0 baja 1 planta
El ascensor 0 deja a la persona 25
El ascensor 0 coge a la persona 20
El ascensor 0 sube 2 plantas
El ascensor 0 deja a la persona 20
El ascensor 0 coge a la persona 17
El ascensor 0 baja 2 plantas
El ascensor 0 coge a la persona 25
El ascensor 0 baja 1 planta
El ascensor 0 deja a la persona 25
El ascensor 0 coge a la persona 9
El ascensor 0 baja 3 plantas
El ascensor 0 deja a la persona 17
El ascensor 0 coge a la persona 24
El ascensor 0 baja 36 plant

In [80]:
print("Nodos analizados: " + str(problem.analizados))
print("Coste secuencial: " + str(coste_acciones(acciones)))
print("Tiempo de búsqueda: " + str(end-start) + " s")

Nodos analizados: 246
Coste secuencial: 771
Tiempo de búsqueda: 0.06947588920593262 s


#### Conclusiones

La definición de las acciones, muy restrictiva gracias a considerar los invariantes implícitos del problema, junto con el control de nodos repetidos del algoritmo, nos ha permitido conseguir una versión del problema que nos lo solucione para una cantidad numerosa de personas en un tiempo razonable. Notemos que no tenemos que indicar al problema el número de plantas a considerar y por lo tanto, podemos poner a cada persona en una planta arbitraria.